In [1]:
!pip install transformers datasets

In [2]:
from datasets import load_dataset
import numpy as np

In [3]:
raw_datasets=load_dataset('glue','rte')

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [5]:
raw_datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'not_entailment'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [6]:
raw_datasets['train']['sentence1'][:10]

['No Weapons of Mass Destruction Found in Iraq Yet.',
 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
 'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.',
 "A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keyworth, Nottinghamshire, on 30 October 1983 when she disappeared. Her body was later found i

In [30]:
# checkpoint='distilbert-base-cased'
checkpoint="bert-base-uncased"

In [31]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments

In [32]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [33]:
tokenizer(
    raw_datasets['train']['sentence1'][0],
    raw_datasets['train']['sentence2'][0]
)

{'input_ids': [101, 2053, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 2664, 1012, 102, 4255, 1997, 3742, 6215, 2179, 1999, 5712, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [34]:
result=_

In [35]:
#disrilbert dont use token_type_ids
result.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [36]:
tokenizer.decode(result['input_ids'])

'[CLS] no weapons of mass destruction found in iraq yet. [SEP] weapons of mass destruction found in iraq. [SEP]'

In [37]:
model=AutoModelForSequenceClassification.from_pretrained(
    checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
pip install accelerate -U

In [38]:
training_args=TrainingArguments(
    output_dir='training_dir',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=150,#otherwise, 'no log' will appear under training loss
)

In [39]:
from datasets import load_metric

In [40]:
metrics=load_metric("glue","rte")

In [41]:
from sklearn.metrics import f1_score

In [42]:
def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels
  predictions=np.argmax(logits,axis=-1)
  acc=np.mean(predictions==labels)
  f1=f1_score(labels,predictions)
  return {'accuracy':acc,'f1':f1}

In [43]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence1'],batch['sentence2'],truncation=True)

In [44]:
tokenized_datasets=raw_datasets.map(tokenize_fn,batched=True)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [45]:
trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [29]:
trainer.train()  #Distill bert

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.700000,0.693807,0.490975,0.629921
2,0.635900,0.721088,0.537906,0.581699
3,0.400300,0.962502,0.577617,0.548263
4,0.208900,1.501983,0.588448,0.580882
5,0.120700,1.880904,0.581227,0.539683


TrainOutput(global_step=780, training_loss=0.3998081085009453, metrics={'train_runtime': 207.2207, 'train_samples_per_second': 60.081, 'train_steps_per_second': 3.764, 'total_flos': 544524318051096.0, 'train_loss': 0.3998081085009453, 'epoch': 5.0})

In [46]:
trainer.train()#Bert

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.659100,0.643227,0.624549,0.570248
2,0.476500,0.675278,0.685921,0.678967
3,0.225600,0.931631,0.700361,0.700361
4,0.138100,1.236608,0.725632,0.724638
5,0.047100,1.464802,0.711191,0.692308


TrainOutput(global_step=780, training_loss=0.29868649198458747, metrics={'train_runtime': 402.7172, 'train_samples_per_second': 30.915, 'train_steps_per_second': 1.937, 'total_flos': 1049544860943240.0, 'train_loss': 0.29868649198458747, 'epoch': 5.0})